<a href="https://colab.research.google.com/github/ahsank/StockML/blob/main/StockML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -q yahoo_fin

In [10]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print


Mounted at /content/drive
/content/drive/MyDrive/colab/results/etf-7b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/etf-7b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/etf-7b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/etf-7b-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [6]:
import numpy as np
import pandas as pd

In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

pd.options.display.max_columns = None

In [8]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '10b', tickers, True)

In [15]:
addAlloc(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
21,DXCM,2.05,0.96,1210.90,1055.97,72.38,82.16,0.14,0.98,0.74,745.12,911.90,94.93,0.31,1.45,0.72,821.88,1027.24,67.00,-0.07,9.314286
14,DASH,3.04,0.95,1199.73,1032.99,151.98,171.31,0.13,1.84,0.78,991.58,1143.05,180.21,0.19,1.57,0.77,1286.71,999.71,138.72,-0.09,9.115385
23,ENPH,5.74,0.93,1958.20,2634.65,91.64,125.90,0.37,2.64,0.71,2282.71,2066.96,146.68,0.60,2.79,0.81,2120.51,2547.63,91.19,-0.00,9.110811
18,DOCS,1.15,0.94,274.63,481.42,43.50,49.51,0.14,0.69,0.73,257.72,453.80,49.47,0.14,0.55,0.83,390.16,509.96,39.55,-0.09,8.971429
10,CFLT,1.20,0.96,405.75,547.37,22.45,23.79,0.06,0.70,0.78,503.90,597.13,24.12,0.07,0.60,0.80,312.08,565.20,19.42,-0.13,8.933333
29,GFS,1.10,0.94,261.81,421.03,40.43,44.37,0.10,0.56,0.62,167.63,284.82,48.61,0.20,0.41,0.78,337.13,334.99,39.64,-0.02,8.800000
44,MBLY,0.79,0.92,107.72,233.46,12.79,15.26,0.19,0.56,0.66,153.74,176.86,16.75,0.31,0.50,0.76,120.92,246.46,12.45,-0.03,8.778947
84,WOLF,2.69,0.91,779.66,1029.74,16.01,20.57,0.28,1.59,0.57,451.52,824.37,22.70,0.42,1.23,0.79,804.41,1130.56,15.02,-0.06,8.778571
1,ACLS,2.59,0.90,1107.84,849.27,92.26,120.86,0.31,1.49,0.80,1388.14,862.73,130.92,0.42,1.47,0.83,1161.16,1164.60,97.00,0.05,8.677419
3,AMBA,2.62,0.94,1172.70,1166.44,58.26,63.05,0.08,1.92,0.73,1225.57,1003.70,61.65,0.06,1.25,0.76,881.08,973.07,52.95,-0.09,8.650000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '7b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
46,XNTK,1.66,0.98,866.49,766.13,191.95,174.58,-0.09,0.97,0.83,970.66,756.01,194.19,0.01,1.19,0.67,781.89,570.50,169.05,-0.12,9.577778
34,XLB,0.58,0.98,375.82,220.82,94.07,91.60,-0.03,0.30,0.80,457.28,310.25,95.30,0.01,0.35,0.72,333.80,231.70,90.07,-0.04,9.133333
28,VNQ,0.57,0.96,344.73,341.10,97.79,104.88,0.07,0.28,0.80,435.53,300.05,100.57,0.03,0.30,0.70,384.78,211.47,99.11,0.01,9.028571
42,XLU,0.50,0.95,272.51,198.18,80.08,74.25,-0.07,0.24,0.84,351.21,227.77,76.26,-0.05,0.23,0.65,241.17,94.40,70.29,-0.12,8.785714
3,CIBR,0.45,0.94,211.67,140.39,59.30,53.80,-0.09,0.30,0.73,211.08,132.44,60.65,0.02,0.25,0.71,256.62,199.17,53.74,-0.09,8.733333
41,XLRE,0.31,0.97,201.02,137.73,44.68,45.86,0.03,0.16,0.81,188.71,121.49,45.74,0.02,0.16,0.63,139.81,118.56,43.89,-0.02,8.700000
31,VUG,2.24,0.95,1489.90,1009.49,380.89,357.59,-0.06,1.41,0.79,1139.53,772.55,382.84,0.01,1.51,0.70,1260.49,1097.45,346.09,-0.09,8.666667
19,MGK,2.11,0.93,1123.67,843.59,319.32,285.75,-0.11,1.07,0.83,1269.86,743.84,322.00,0.01,1.19,0.83,1576.50,1382.96,293.84,-0.08,8.663636
16,IWO,2.42,0.96,1099.42,1133.34,284.01,272.06,-0.04,1.20,0.76,1099.86,1089.75,289.19,0.02,1.20,0.74,1377.57,1218.43,269.48,-0.05,8.600000
2,ARKW,1.60,0.94,514.51,891.02,84.03,89.54,0.07,1.21,0.64,341.15,537.93,88.33,0.05,1.28,0.79,801.91,1050.43,69.49,-0.17,8.542857


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
lossfn = findata.Huber()
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'DPSI', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON',
            'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombinedVola(tickers3, 'vols-8b', mod, True, loss=lossfn)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
25,GOGO,0.29,0.95,134.12,153.34,7.24,8.32,0.15,0.31,0.66,163.99,142.46,8.13,0.12,0.18,0.73,68.62,93.26,6.85,-0.05,5.254902
36,LAZR,0.46,0.90,85.33,216.86,0.91,1.37,0.51,0.74,0.49,-11.82,104.34,1.37,0.51,0.30,0.68,99.08,219.98,0.96,0.05,5.098039
34,JOBY,0.18,0.92,57.28,88.64,5.22,6.50,0.25,0.26,0.56,46.15,91.44,5.77,0.10,0.13,0.72,54.46,93.53,4.93,-0.05,5.091765
3,AMSC,0.50,0.90,164.02,190.06,22.98,30.19,0.31,0.50,0.56,191.42,161.51,30.71,0.34,0.35,0.72,116.30,130.84,16.25,-0.29,4.971537
27,GRPN,0.83,0.89,191.61,307.57,9.96,13.16,0.32,0.94,0.70,188.22,242.38,13.57,0.36,0.62,0.67,103.50,269.32,9.34,-0.06,4.891544
74,YEXT,0.24,0.90,61.14,106.87,6.72,7.78,0.16,0.33,0.59,47.03,68.77,7.90,0.18,0.17,0.74,49.56,85.68,6.19,-0.08,4.669118
20,CXM,0.28,0.94,82.46,135.85,7.73,8.18,0.06,0.32,0.64,67.17,77.48,9.33,0.21,0.21,0.66,48.49,103.06,7.45,-0.04,4.529412
31,INDI,0.26,0.90,86.63,108.92,4.17,4.71,0.13,0.21,0.64,67.07,77.71,4.62,0.11,0.14,0.74,70.39,93.01,3.77,-0.09,4.524887
18,CLOV,0.29,0.83,49.09,108.79,2.82,4.06,0.44,0.44,0.45,48.69,105.73,6.02,1.14,0.10,0.64,28.87,80.85,2.07,-0.26,4.495989
64,SLE,2.41,0.76,521.21,863.30,0.65,1.21,0.88,2.46,0.52,915.71,1119.32,3.51,4.44,1.27,0.75,622.75,677.78,0.97,0.50,4.197861


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombinedVola(tickers4, 'mcap-8b', mod4, False)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
6,BA,3.72,0.96,1195.28,1609.71,156.32,176.98,0.13,2.57,0.65,985.34,1151.74,168.69,0.08,1.89,0.72,835.87,1119.20,130.00,-0.17,6.549451
2,ADBE,8.44,0.95,4846.85,3562.30,515.48,563.26,0.09,5.12,0.73,3141.35,2700.64,543.30,0.05,5.33,0.67,2845.61,2495.04,501.29,-0.03,6.230159
11,CRM,3.61,0.95,1602.30,1592.33,276.64,255.47,-0.08,2.45,0.74,1298.56,943.42,278.62,0.01,2.48,0.71,1092.29,1277.39,242.16,-0.12,6.160714
33,QCOM,2.16,0.94,1166.71,877.31,170.13,155.44,-0.09,1.63,0.72,856.04,769.76,184.31,0.08,1.32,0.73,901.70,834.28,157.37,-0.08,6.047619
36,TSLA,6.45,0.93,3725.14,2384.86,260.46,231.35,-0.11,4.27,0.68,2785.49,2422.26,253.77,-0.03,4.01,0.72,2389.93,2341.48,256.33,-0.02,6.006494
3,AMD,2.71,0.95,1301.09,965.56,164.35,154.18,-0.06,2.10,0.64,952.01,575.15,177.13,0.08,1.52,0.78,1282.34,919.43,135.89,-0.17,5.952381
22,IBM,2.04,0.93,744.11,505.74,220.84,197.88,-0.10,1.63,0.72,865.84,211.19,213.71,-0.03,1.08,0.61,489.12,184.94,199.93,-0.09,5.942857
10,CDNS,3.72,0.94,1512.21,1033.02,274.13,289.49,0.06,2.35,0.77,1415.51,881.66,291.63,0.06,1.99,0.63,1066.94,662.10,255.19,-0.07,5.714286
31,ORCL,1.85,0.91,712.35,399.71,168.74,150.31,-0.11,1.50,0.78,886.35,400.00,171.42,0.02,1.06,0.69,508.97,164.85,146.58,-0.13,5.681818
4,AMZN,2.47,0.93,1120.41,976.24,187.97,174.02,-0.07,1.49,0.78,1007.18,828.11,186.03,-0.01,1.39,0.70,604.04,1023.01,173.70,-0.08,5.642857


In [ ]:
findata.EPOCHS=200
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '10a', tickers5, True)

In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
23,FQVLF,0.65,0.93,229.00,278.26,13.78,16.98,0.23,0.48,0.72,160.62,214.45,16.38,0.19,0.39,0.79,229.23,299.45,13.30,-0.03,6.034161
56,STLA,0.34,0.83,97.20,93.11,13.32,17.96,0.35,0.25,0.72,107.37,88.15,17.32,0.30,0.18,0.73,112.17,99.90,13.80,0.04,4.957143
45,PAX,0.20,0.92,58.88,73.44,11.00,11.91,0.08,0.14,0.72,49.86,82.27,11.63,0.06,0.08,0.77,87.67,84.13,10.69,-0.03,4.571429
33,MGA,1.04,0.91,401.52,413.14,41.08,44.83,0.09,0.64,0.78,343.91,437.20,45.45,0.11,0.78,0.73,367.85,448.95,39.32,-0.04,4.500000
52,ORAN,0.09,0.94,45.35,31.64,10.74,11.27,0.05,0.06,0.71,34.71,23.72,11.64,0.08,0.05,0.78,41.37,40.40,10.91,0.02,4.314286
51,SFTBY,0.63,0.93,183.84,249.56,29.86,31.76,0.06,0.39,0.71,193.07,144.26,31.44,0.05,0.36,0.78,180.21,175.92,25.47,-0.15,4.309524
3,BAK,0.35,0.93,105.49,160.14,7.38,7.78,0.05,0.31,0.72,106.90,113.27,7.49,0.02,0.22,0.82,132.10,161.49,6.43,-0.13,3.842857
4,BASFY,0.19,0.94,65.83,88.35,13.24,13.58,0.03,0.11,0.72,44.06,67.23,13.35,0.01,0.11,0.72,59.90,70.84,12.15,-0.08,2.714286
37,LVMUY,2.55,0.92,937.19,748.10,146.70,152.10,0.04,1.72,0.64,578.77,662.60,152.19,0.04,1.28,0.71,746.66,548.61,135.44,-0.08,2.571429
25,GLOB,5.64,0.88,1722.62,1827.03,193.45,205.48,0.06,3.35,0.68,1251.72,1673.24,202.87,0.05,2.56,0.82,1929.21,2459.61,192.50,-0.00,2.285714


In [ ]:
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '8a', tickers6, False)

In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
3,ECH,0.29,0.96,138.96,155.62,26.52,24.42,-0.08,0.19,0.79,129.13,122.16,26.68,0.01,0.21,0.66,96.61,100.67,24.68,-0.07,9.100000
15,EWZ,0.30,0.95,158.10,157.80,29.41,27.16,-0.08,0.19,0.76,182.47,184.27,30.12,0.02,0.17,0.74,184.15,142.99,26.80,-0.09,8.875000
17,ILF,0.28,0.94,135.46,94.45,26.05,23.56,-0.10,0.14,0.76,141.48,101.59,25.34,-0.03,0.17,0.65,87.15,51.80,22.90,-0.12,8.800000
2,FLN,0.17,0.93,72.74,76.31,17.90,16.68,-0.07,0.11,0.81,96.35,85.14,17.60,-0.02,0.08,0.74,98.09,83.73,16.53,-0.08,8.300000
12,EWT,0.52,0.88,171.52,156.65,53.78,47.58,-0.12,0.35,0.75,173.06,133.64,53.19,-0.01,0.47,0.70,169.77,153.10,46.28,-0.14,7.800000
14,EWY,0.72,0.92,224.74,332.33,62.70,59.87,-0.05,0.45,0.71,191.58,253.62,65.22,0.04,0.56,0.68,184.21,271.46,58.69,-0.06,7.600000
4,EPHE,0.32,0.92,116.84,104.13,29.31,27.83,-0.05,0.22,0.70,66.90,92.67,28.71,-0.02,0.23,0.66,75.92,69.74,26.66,-0.09,7.600000
21,TUR,0.43,0.91,234.84,121.72,34.40,32.66,-0.05,0.37,0.76,184.25,136.48,35.89,0.04,0.36,0.72,145.69,167.57,34.09,-0.01,7.300000
8,EWJ,0.58,0.91,200.58,161.05,71.88,67.98,-0.05,0.39,0.73,193.03,128.81,71.47,-0.01,0.46,0.67,195.14,172.24,66.10,-0.08,7.300000
1,EWD,0.39,0.92,154.97,174.09,42.23,40.38,-0.04,0.21,0.80,212.26,145.58,42.53,0.01,0.27,0.63,133.39,142.93,39.04,-0.08,7.200000


In [ ]:
findata.EPOCHS=200
review = [ 'IFF', 'BLK']
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']
df7 = run_model('val', '9a', tickers7, False)

In [ ]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
56,MMM,1.24,0.96,540.61,528.49,134.75,143.44,0.06,0.85,0.83,672.78,465.39,144.59,0.07,0.74,0.67,460.26,293.91,124.29,-0.08,8.933333
34,DXC,0.60,0.93,180.56,213.31,20.02,22.79,0.14,0.32,0.72,172.42,209.31,23.13,0.16,0.26,0.75,228.36,281.08,18.98,-0.05,8.800000
67,PETS,0.37,0.91,86.21,175.85,3.90,4.60,0.18,0.34,0.61,143.39,247.73,5.79,0.48,0.22,0.86,193.48,273.99,3.53,-0.09,8.600000
60,NTR,1.05,0.91,431.54,337.24,49.05,54.25,0.11,0.67,0.72,355.94,354.20,53.39,0.09,0.62,0.71,437.57,422.65,46.90,-0.04,8.281818
59,NFG,0.43,0.97,259.21,163.39,60.36,61.66,0.02,0.36,0.78,273.53,190.79,65.85,0.09,0.28,0.68,297.68,186.50,58.52,-0.03,8.200000
64,PARA,0.79,0.87,232.21,494.71,10.35,12.91,0.25,0.70,0.66,196.58,327.53,12.71,0.23,0.47,0.79,187.83,462.39,10.49,0.01,8.180000
45,HIW,0.34,0.91,136.49,170.55,33.61,36.58,0.09,0.24,0.77,174.54,184.23,37.42,0.11,0.20,0.71,166.80,168.37,31.39,-0.07,8.100000
12,BGFV,0.53,0.86,161.82,130.45,2.08,2.50,0.20,0.33,0.70,166.68,200.13,2.83,0.36,0.23,0.73,108.71,144.49,1.99,-0.05,7.900000
95,WHR,2.10,0.92,717.93,1008.68,105.11,111.49,0.06,1.57,0.78,1003.19,1030.64,107.20,0.02,1.17,0.68,785.91,688.13,95.87,-0.09,7.866667
58,MPC,2.17,0.91,1039.00,484.28,164.26,175.78,0.07,1.65,0.82,1002.13,588.20,170.02,0.04,1.23,0.63,817.64,122.01,137.69,-0.16,7.814286


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '10a', tickers8, True)

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
7,SLV,0.35,0.93,112.37,116.20,28.80,25.66,-0.11,0.21,0.73,128.75,121.41,28.63,-0.01,0.22,0.70,104.75,79.26,26.56,-0.08,8.663636
3,FXY,0.54,0.94,80.54,223.52,61.92,66.97,0.08,0.35,0.69,69.87,142.59,64.13,0.04,0.46,0.76,58.23,147.88,62.30,0.01,8.650000
11,USO,1.58,0.93,467.48,321.87,77.49,71.76,-0.07,1.08,0.80,514.98,352.37,78.88,0.02,1.07,0.63,394.04,221.24,66.89,-0.14,8.300000
14,WEAT,0.14,0.89,45.86,51.51,5.38,6.03,0.12,0.11,0.67,41.94,31.12,5.73,0.06,0.09,0.73,28.69,37.21,5.38,-0.00,7.983333
8,SOYB,0.26,0.89,81.26,82.30,22.04,20.05,-0.09,0.20,0.70,64.17,70.78,22.68,0.03,0.21,0.73,91.21,76.18,21.02,-0.05,7.677778
4,GLD,1.89,0.89,626.59,398.00,245.47,232.53,-0.05,1.08,0.76,773.07,387.84,245.51,0.00,1.58,0.68,392.45,152.94,228.84,-0.07,6.700000
10,TLT,1.21,0.90,270.71,456.51,93.70,97.20,0.04,0.93,0.70,143.01,337.05,96.19,0.03,0.99,0.78,266.57,453.21,91.59,-0.02,6.500000
0,FXB,0.68,0.89,207.85,186.45,125.60,122.04,-0.03,0.36,0.72,147.02,157.09,127.25,0.01,0.50,0.66,142.62,98.59,123.84,-0.01,5.233333
1,FXE,0.46,0.90,117.19,140.18,100.93,102.88,0.02,0.32,0.70,96.75,130.55,103.26,0.02,0.43,0.71,87.89,144.98,100.69,-0.00,4.000000
2,FXF,0.45,0.94,167.47,133.19,103.73,103.19,-0.01,0.30,0.73,159.20,124.73,105.51,0.02,0.31,0.78,167.75,151.55,102.08,-0.02,3.400000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '9a', tickers9, False)

In [13]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
7,FIL-USD,0.64,0.85,61.69,130.63,3.72,4.24,0.14,0.75,0.71,59.00,88.04,4.81,0.29,0.30,0.70,41.80,107.11,3.49,-0.06,5.000000
4,DOGE-USD,0.01,0.82,1.14,1.49,0.14,0.11,-0.20,0.01,0.66,1.66,1.30,0.14,-0.02,0.00,0.68,1.02,0.97,0.11,-0.24,4.957143
11,MATIC-USD,0.07,0.76,6.23,11.71,0.37,0.55,0.49,0.07,0.57,1.15,8.83,0.79,1.12,0.03,0.75,6.61,12.33,0.41,0.11,4.938776
6,ETH-USD,89.36,0.84,19661.38,24301.07,2642.96,2305.92,-0.13,69.51,0.68,14271.57,17925.51,2698.59,0.02,40.71,0.70,15021.52,24195.53,2289.18,-0.13,4.769231
5,DOT-USD,0.52,0.86,61.69,98.15,4.42,4.87,0.10,0.54,0.65,23.53,64.18,5.00,0.13,0.31,0.75,61.71,146.19,4.10,-0.07,4.742857
1,AVAX-USD,2.85,0.86,359.40,629.52,27.98,30.54,0.09,2.36,0.60,250.41,359.50,33.07,0.18,1.13,0.72,307.27,494.53,24.16,-0.14,4.587302
14,TON-USD,0.13,0.82,15.83,25.40,1.16,1.27,0.10,0.23,0.73,62.03,68.00,1.54,0.34,0.07,0.80,21.65,37.50,1.00,-0.14,4.057143
16,XRP-USD,0.02,0.87,4.23,6.41,0.54,0.57,0.05,0.02,0.71,5.57,6.86,0.57,0.06,0.01,0.76,5.21,5.49,0.54,-0.01,3.614286
3,BCH-USD,14.22,0.81,3252.97,3283.06,363.13,333.48,-0.08,16.93,0.67,3874.57,3141.59,379.57,0.05,5.71,0.73,2506.47,3101.09,294.15,-0.19,3.410714
0,ADA-USD,0.04,0.76,3.32,3.47,0.35,0.39,0.10,0.03,0.66,2.18,5.30,0.40,0.15,0.01,0.70,3.16,4.51,0.33,-0.06,3.028571


In [16]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
ipos-10b-adjclose-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
ipos-10b-high-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
ipos-10b-low-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras

sent 28,834,360 bytes  received 73 bytes  57,668,866.00 bytes/sec
total size is 86,477,630  speedup is 3.00
